### Wide&Deep 模型


In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import re

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, Concatenate, Dropout, Input, Layer
from tensorflow.keras.regularizers import l2

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import os

In [2]:

def sparseFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}

def denseFeature(feat):
    """
    create dictionary for dense feature
    :param feat: dense feature name
    :return:
    """
    return {'feat': feat}


def create_criteo_dataset(file_train, file_test, embed_dim=8, read_part=False, sample_num=100000, test_size=0.2):

    # 训练数据
    df_train = pd.read_csv(file_train)
    # 测试数据
    df_apply_new = pd.read_csv(file_test)
    # 合并训练集，验证集
    data_df = pd.concat([df_train, df_apply_new], axis=0, ignore_index=True)
    data_df['label'] = data_df['label'].fillna(str(-1))

    
    ### 增加几列
    # 增加六个维度
    data_df['province_sum'] = df_train.groupby(['province'])['label'].transform('sum')
    val = data_df['province'].value_counts()
    data_df['province_pre'] = data_df['province_sum'] / data_df['province'].apply(lambda x: val[x])

    data_df['city_sum'] = df_train.groupby(['city'])['label'].transform('sum')
    val = data_df['city'].value_counts()
    data_df['city_pre'] = data_df['city_sum'] / data_df['city'].apply(lambda x: val[x])

    data_df['model_sum'] = df_train.groupby(['model'])['label'].transform('sum')
    val = data_df['model'].value_counts()
    data_df['model_pre'] = data_df['model_sum'] / data_df['model'].apply(lambda x: val[x])
    
    
    
    # 稀疏特征 label 类型的特征 不需要归一化，非连续
    sparse_features = ['gender', 'province', 'city', 'make', 'model']
    # 稠密特征 需要归一化 连续性的特征
    dense_features = ['age', 'province_sum', 'city_sum', 'model_sum']


    def clean_data(string):
        # 对数据清洗
        string = re.sub(r"[^0-9()]", "", string)
        return string.strip().lower()


    # ==============Age ===================
    # 处理Age
    # 缺失值填充
    data_df['age'] = data_df['age'].fillna(-1)
    a = data_df['age'].copy()
    a = a.apply(lambda x: str(x).lower())
    # 统一字符类型转化成str()
    a = a.apply(lambda x: clean_data(x))
    data_df['age'] = a
    data_df['age'] = data_df['age'].astype('int')  # 转换数据类型为 int 类型


    # ==============appid_num ===================
    appid_num = data_df['appid']
    def get_appid_num(string):
        # 对数据清洗
        string = string.split(',')
        return len(string)

    appid_num = appid_num.apply(lambda x: get_appid_num(x))
    data_df['appid_num'] = appid_num
    dense_features = dense_features + ['appid_num']


    data_df[sparse_features] = data_df[sparse_features].fillna('-1')
    data_df[dense_features] = data_df[dense_features].fillna(0)

    
    
    # ==============gender ===================
    data_df['gender'] = data_df['gender'].astype('str')  # 转换数据类型为 int 类型


    for feat in sparse_features:
        le = LabelEncoder()
        data_df[feat] = le.fit_transform(data_df[feat])

        

    # ====================================================
    mms = MinMaxScaler(feature_range=(0, 1))
    data_df[dense_features] = mms.fit_transform(data_df[dense_features])

    
    dense_features = dense_features + ['province_pre', 'city_pre', 'model_pre']
    # 统计dense_features、sparse_features每个特征的个数和
    feature_columns = [[denseFeature(feat) for feat in dense_features]] + \
                      [[sparseFeature(feat, len(data_df[feat].unique()), embed_dim=embed_dim)
                        for feat in sparse_features]]

    # 划分训练集和测试集
    """
    ### 本次案例中：将所有的样本作为训练集。
    ### 使用全部的样本作为训练集，通过交叉验证的方法划分为：测试集+验证集
    """
    data_df_1 = data_df[:len(df_train)]  # data.click != -1的样本为训练的样本集合
    train, test = train_test_split(data_df_1, test_size=test_size)

    train_X = [train[dense_features].values.astype('int32'), train[sparse_features].values.astype('int32')]
    train_y = train['label'].values.astype('int32')
    test_X = [test[dense_features].values.astype('int32'), test[sparse_features].values.astype('int32')]
    test_y = test['label'].values.astype('int32')


    # 划分需要预测的样本集
    data_df_2 = data_df[-len(df_apply_new):]  # data.click == -1的样本为需要预测的样本集合
    total_test = [data_df_2[dense_features].values.astype('int32'), data_df_2[sparse_features].values.astype('int32')]

    return feature_columns, (train_X, train_y), (test_X, test_y), total_test

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# you can modify your file path
file_train = '../data/train.csv'
file_test = '../data/apply_new.csv'


embed_dim = 12
read_part = True
sample_num = 6000000
test_size = 0.2

# ========================== Create dataset =======================
feature_columns, train, test, vail = create_criteo_dataset(file_train=file_train,
                                                           file_test=file_test,
                                                           embed_dim=embed_dim,
                                                           read_part=read_part,
                                                           sample_num=sample_num,
                                                           test_size=test_size)

F:\Python\1\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
feature_columns

[[{'feat': 'age'},
  {'feat': 'province_sum'},
  {'feat': 'city_sum'},
  {'feat': 'model_sum'},
  {'feat': 'appid_num'},
  {'feat': 'province_pre'},
  {'feat': 'city_pre'},
  {'feat': 'model_pre'}],
 [{'feat': 'gender', 'feat_num': 3, 'embed_dim': 12},
  {'feat': 'province', 'feat_num': 34, 'embed_dim': 12},
  {'feat': 'city', 'feat_num': 316, 'embed_dim': 12},
  {'feat': 'make', 'feat_num': 1549, 'embed_dim': 12},
  {'feat': 'model', 'feat_num': 129, 'embed_dim': 12}]]

In [6]:
class Linear(Layer):
    """
    Linear Part
    """
    def __init__(self):
        super(Linear, self).__init__()
        self.dense = Dense(1, activation=None)

    def call(self, inputs, **kwargs):
        result = self.dense(inputs)
        return result


class DNN(Layer):
    """
	Deep Neural Network
	"""

    def __init__(self, hidden_units, activation='relu', dropout=0.):
        """
		:param hidden_units: A list. Neural network hidden units.
		:param activation: A string. Activation function of dnn.
		:param dropout: A scalar. Dropout number.
		"""
        super(DNN, self).__init__()
        self.dnn_network = [Dense(units=unit, activation=activation) for unit in hidden_units]
        self.dropout = Dropout(dropout)

    def call(self, inputs, **kwargs):
        x = inputs
        for dnn in self.dnn_network:
            x = dnn(x)
        x = self.dropout(x)
        return x


class WideDeep(tf.keras.Model):
    def __init__(self, feature_columns, hidden_units, activation='relu',
                 dnn_dropout=0., embed_reg=1e-4):
        """
        Wide&Deep
        :param feature_columns: A list. dense_feature_columns + sparse_feature_columns
        :param hidden_units: A list. Neural network hidden units.
        :param activation: A string. Activation function of dnn.
        :param dnn_dropout: A scalar. Dropout of dnn.
        :param embed_reg: A scalar. The regularizer of embedding.
        """
        super(WideDeep, self).__init__()
        self.dnn_network = DNN(hidden_units, activation, dnn_dropout)
        self.linear = Linear()
        self.final_dense = Dense(1, activation=None)
        self.dense_feature_columns, self.sparse_feature_columns = feature_columns
        self.embed_layers = {
            'embed_' + str(i): Embedding(input_dim=feat['feat_num'],
                                         input_length=1,
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=l2(embed_reg))
            for i, feat in enumerate(self.sparse_feature_columns)
        }


    def call(self, inputs, **kwargs):
        dense_inputs, sparse_inputs = inputs
        sparse_embed = tf.concat([self.embed_layers['embed_{}'.format(i)](sparse_inputs[:, i])
                                  for i in range(sparse_inputs.shape[1])], axis=-1)
        dense_inputs = tf.cast(dense_inputs, dtype=tf.float32)  # dense_inputs 类型转化成float
        x = tf.concat([sparse_embed, dense_inputs], axis=-1)

        # Wide
        wide_out = self.linear(dense_inputs)
        # Deep
        deep_out = self.dnn_network(x)
        deep_out = self.final_dense(deep_out)
        # out
        outputs = tf.nn.sigmoid(0.5 * (wide_out + deep_out))
        return outputs

    def summary(self, **kwargs):
        dense_inputs = Input(shape=(len(self.dense_feature_columns),), dtype=tf.int32)
        sparse_inputs = Input(shape=(len(self.sparse_feature_columns),), dtype=tf.int32)
        keras.Model(inputs=[dense_inputs, sparse_inputs],
                    outputs=self.call([dense_inputs, sparse_inputs])).summary()

In [7]:
dnn_dropout = 0.5
hidden_units = [256, 128, 64]

learning_rate = 0.001
batch_size = 256
epochs = 20

# ========================== Create dataset =======================
train_X, train_y = train
test_X, test_y = test

# ============================Build Model==========================
model = WideDeep(feature_columns, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None,)]            0           input_2[0][0]                    
______________________________________________________________________________________________

In [8]:
# ============================Compile============================
model.compile(loss=binary_crossentropy, 
              optimizer=Adam(learning_rate=learning_rate),
              metrics=[AUC()])
# ==============================Fit==============================
history = model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y)[1])

Train on 216000 samples, validate on 24000 samples
Epoch 1/20


F:\Python\1\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


216000/216000 [==============================] - 5s 22us/sample - loss: 0.6203 - auc: 0.6988 - val_loss: 0.6086 - val_auc: 0.7111
Epoch 2/20
216000/216000 [==============================] - 4s 16us/sample - loss: 0.6080 - auc: 0.7162 - val_loss: 0.6074 - val_auc: 0.7136
Epoch 3/20
216000/216000 [==============================] - 4s 16us/sample - loss: 0.6069 - auc: 0.7178 - val_loss: 0.6074 - val_auc: 0.7140
60000/1 [===================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

test AUC: 0.718274


In [9]:
prediction = model.predict(vail)


In [18]:
prediction

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [10]:
prediction[:, 0][:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [11]:
a = np.round(prediction[:, 0])

In [12]:
# 读入文件并写入预测值
label_submission = pd.read_csv('../data/submit_sample.csv')
label_submission.head()

,user_id,category_id
0,1400001,0
1,1400002,1
2,1400003,0
3,1400004,0
4,1400005,0


In [13]:
len(label_submission)

100000

In [14]:
len(a)

100000

In [15]:
label_submission['category_id']=a.astype(int)

In [16]:
label_submission

,user_id,category_id
0,1400001,0
1,1400002,0
2,1400003,0
3,1400004,0
4,1400005,0
...,...,...
99995,1499996,0
99996,1499997,0
99997,1499998,0
99998,1499999,0


In [17]:
label_submission.to_csv("../submission/submission_Wide_deep_v4.csv",index=False)